In [2]:
from __future__ import print_function
%load_ext autoreload
%autoreload 2
from pixell import enmap, utils
from pixell import reproject
from pixell import enplot
from pyuvdata import UVData
import numpy as np
import matplotlib.pyplot as plt
import PIL
from glob import glob
from astropy.io import fits

In [3]:
# Convenient wrapper for displaying enmaps
def eshow(x,**kwargs): enplot.show(enplot.plot(x,**kwargs))

In [4]:
path = '/Users/jaguirre/Data/IDR2CASA/'
filename = path+'zen.2458098.22991.HH.calibrated.uvh5.image.image.fits'
fitsfiles = glob(path+'zen.*.HH.calibrated.uvh5.image.image.fits').sort()
muellermodel = '/home/gonzalo/Desktop/workstation/muellermatrixsim.csv'

In [5]:
# Define some dimensions
fitsfile = fitsfiles[0]
# Read the map
imap = enmap.read_map(fitsfile,)
# For some reason, pixell makes it really hard to get at the actual values in the WCS header
hdu = fits.open(fitsfile)
# Define the size of the pixels in the uber-map as a fraction of the size of the original map
pix_fac = 2.
pixsize_new = np.abs(imap.wcs.pixel_scale_matrix[0][0])*60./pix_fac * utils.arcmin
#print(pixsize_new) # radians?

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# Define the master map
# Define its shape
shape_band, wcs_band = enmap.band_geometry(np.radians([-60,-10]), 
                                           res=pixsize_new, shape=shape_fullsky, proj='car')
#print(shape_band)
#print(wcs_band)
mastermap = enmap.enmap(np.zeros(shape_band), wcs=wcs_band)

In [ ]:
for fitsfile in fitsfiles:
    print(fitsfile)
    imap = enmap.read_map(fitsfile,)
    # For now, extract Stokes I
    stokesI = imap[0,0,:,:]
    # Also get the necessary fits information
    hdu = fits.open(fitsfile)
    # Extract a postage stamp centered on the center of the image
    dec0, ra0 = np.deg2rad(np.array((hdu[0].header['CRVAL2'],hdu[0].header['CRVAL1'])))
    width = np.deg2rad(20.)
    # [[decfrom,rafrom],[[decto,rato]]
    box = np.array([[dec0-width/2.,ra0-width/2.],[dec0+width/2.,ra0+width/2.]])
    #print(dec,ra,width)
    #print(box)
    stamp = stokesI.submap(box)
    
    
    # project the stamp into the master grid
    stampmap = enmap.project(stamp, shape_band, wcs_band)
    print(stampmap.shape())
    # Accumulate
    #mastermap += stampmap

In [ ]:
eshow(mastermap)

In [ ]:
# write the output back to fits
enmap.write_fits('masterpmap.fits', mastermap)